# Generate stock time series data

In [1]:
import numpy as np
import pandas as pd
import requests
import bokeh.plotting as bk
from bokeh.models import HoverTool, Label, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool

API_URL = 'https://api.iextrading.com/1.0'

## Make request to API and format as DataFrame

In [2]:
res = requests.get(f'{API_URL}/stock/V/chart/5y')

In [3]:
data = res.json()

In [4]:
df = pd.DataFrame(data)

In [5]:
df.sample(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap
178,0.529451,0.199151,1.048,51.0265,2014-05-21,51.1647,"May 21, 14",50.6598,50.7229,2274146,9096584,50.9877
215,0.182148,0.265815,0.339,53.8632,2014-07-15,54.5845,"Jul 15, 14",53.6495,53.8365,2725504,10902016,54.0118


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 12 columns):
change              1259 non-null float64
changeOverTime      1259 non-null float64
changePercent       1259 non-null float64
close               1259 non-null float64
date                1259 non-null object
high                1259 non-null float64
label               1259 non-null object
low                 1259 non-null float64
open                1259 non-null float64
unadjustedVolume    1259 non-null int64
volume              1259 non-null int64
vwap                1259 non-null float64
dtypes: float64(8), int64(2), object(2)
memory usage: 118.1+ KB


## Candlestick chart setup

In [7]:
seqs = np.arange(df.shape[0])
df['seqs'] = pd.Series(seqs)

In [8]:
df['changePercent'] = df['changePercent'].apply(lambda x: str(x)+'%')

In [9]:
df['mid'] = df.apply(lambda x: (x['open'] + x['close']) / 2, axis=1)

In [10]:
df['height'] = df.apply(
    lambda x: x['close'] - x['open'] if x['close'] != x['open'] else 0.001, 
    axis=1)

In [11]:
inc = df.close > df.open
dec = df.close < df.open
w = .3

In [12]:
sourceInc = bk.ColumnDataSource(df.loc[inc])
sourceDec = bk.ColumnDataSource(df.loc[dec])

In [13]:
hover = HoverTool(
    tooltips=[
        ('date', '@date'),
        ('low', '@low'),
        ('high', '@high'),
        ('open', '@open'),
        ('close', '@close'),
        ('percent', '@changePercent'),
    ]
)

In [14]:
TOOLS = [hover, BoxZoomTool(), PanTool(), ZoomInTool(), ZoomOutTool(), ResetTool()]

In [15]:
p = bk.figure(plot_width=1500, plot_height=800, tools=TOOLS, title='Microsoft', toolbar_location='above')

In [16]:
p.xaxis.major_label_orientation = np.pi/4
p.grid.grid_line_alpha=w

In [17]:
descriptor = Label(x=70, y=70, text='Your label goes here')
p.add_layout(descriptor)

In [18]:
p.segment(df.seqs[inc], df.high[inc], df.seqs[inc], df.low[inc], color='green')

bokeh.models.renderers.GlyphRenderer(
    id='84e0d9de-8680-430b-836e-4e49d92b4625',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='4c73de58-6eb3-48cf-b947-f1dff122606d',
        callback=None,
        data={'y0': {'__ndarray__': '5/up8dJVRUBRa5p3nHpFQMPTK2UZkkVAFmpN845LRkA9m1Wfq4VGQOzAOSNK20ZAMlUwKqkrR0DNO07RkWxHQDnWxW006EdAxEKtad4BSEAVjErqBHxHQKTfvg6cY0dAkzoBTYQtR0CNl24Sg2BHQBSuR+F6FEdAIo51cRtNRkAvbqMBvNVGQLByaJHtNEdArkfhehRmR0A/xty1hKRHQLraiv1l/0dAvw6cM6JESEBDrWnecQpIQINRSZ2AjkhAVcGopE7ISEAVjErqBERIQB04Z0Rp50dAlkOLbOcrSEAtIR/0bPZHQPhT46WbHEhAwcqhRbYrSEAoDwu1pjlIQCntDb4wgUhAxm00gLc4SEBjf9k9eXBIQPtcbcX+ekhAbqMBvAWiSEDjpZvEIMhIQE+vlGWI+0hA7C+7Jw+DSECLbOf7qZlIQDojSnuDJ0lA/Knx0k0qSUAgY+5aQt5JQESLbOf7EUpA63O1FftbSkDgnBGlvXlKQP7UeOkmkUpAyxDHurilSkDHuriNBtBKQKd5xyk68kpAyAc9m1XfSkDecYqO5PpKQI51cRsN8EpAkst/SL81S0C2hHzQsxlMQG6jAbwFSkxAArwFEhTnSkCfq63YXx5KQLfRAN4COUpAZapgVFJ/SkBd/kP67dtKQHNoke1890pA/Knx0k1CS0B24JwRpT1LQM6I0t7ga0tAf/s6cM6QS0DyQc9m1a9LQNzXgXNGnEtAJJf/kH5jS0CfPCzUmmZLQECk374OXEtAO99PjZeuS0ASFD/G3OVKQHuDL0ymIktAak3zjlN8S0AIrBxaZDNMQM9m1edqM0pA0m9fB84pSkBlGeJYFy9JQOC+DpwzekhA5dAi2/nWSEARNjy9UmZJQE3zjlN0ZElALv8h/faFSUDT3uALk3lIQN6Th4Vau0hAZvfkYaGuSECNl24SgyBJQHxhMlUwMklAFNBE2PBUSUDpSC7/IbVJQIJzRpT2jklA/kP67evASUDl8h/Sb8dJQLPqc7UVg0lASZ2AJsKeSUBb07zjFJVJQGaIY13cxklAzczMzMwMSkBgdk8eFhJKQNv5fmq8rElAWRe30QDOSUBEi2zn++lJQCL99nXgFEpAhxbZzve7SUDzjlN0JKdJQFHaG3xheklAOiNKe4OPSUB3vp8aL31JQJ+rrdhftklAnzws1JomSkCL/WX35FlKQPtcbcX+akpAiPTb14FDSkC8lpAPelZKQEJg5dAiW0pA8fRKWYYAS0CJQWDl0EpLQO5aQj7o8UpA5IOezaoHS0B/arx0k1BKQNobfGEyFUpAHThnRGnfSUDQ1VbsL6tJQP5l9+RhkUlA6Gor9pelSUA8vVKWIc5JQBPyQc9m3UlAuycPC7UWSkDLoUW28z1KQFpkO99PVUpAPSzUmuaFSkBz1xLyQXdKQJ2AJsKGJ0pA9dvXgXNOSkDu68A5I5pKQPp+arx0O0pA16NwPQp/SkBpb/CFyVxKQP8h/fZ1KEpA0m9fB87RSUDgvg6cM7pJQG6jAbwFCkpAeJyiI7mMSUCVZYhjXdRJQKRwPQrX20lAt2J/2T25SECL/WX35CFJQIbJVMGoVElAWRe30QD2SUB4CyQofkRKQI51cRsNIEpAtTf4wmRiSkCx4emVshxNQPAWSFD8gE1AEFg5tMiWTUBwzojS3pBNQPp+arx0s05AApoIG56GTkAgY+5aQrZOQAIrhxbZZk5APujZrPq8TkAukKD4MaZOQGrecYqOtE5A+u3rwDlzT0AE54wo7cVPQIzbaABvuU9Ao5I6AU3sT0CIY13cRg9QQLTIdr6fIlBAgLdAguIvUEB+HThnRIFPQJZDi2zno09AbjSAt0AeUECjAbwFEhxQQKJFtvP9PFBAQs9m1ediUEDu68A5I0JQQD55WKg11U9AYhBYObQoUEApyxDHuhhPQNXnaiv2p09AEqW9wRemT0DfT42XbpJPQDJ3LSEfVE5ASOF6FK6/T0CZKhiV1EFQQGKh1jTvkFBABaOSOgE9UEDbiv1l94RQQOhqK/aXpVBAO99PjZeqUEAbnl4py7hQQLfRAN4CvVBAwOyePCz4UEDM7snDQplQQPvL7snDdlBAswxxrIt3UECsHFpkO1dQQMGopE5Ag1BA8kHPZtUrUEC3Yn/ZPQ1QQEw3iUFgAVBAtMh2vp8CUEDbiv1l9yRQQL1SliGOTVBANqs+V1sVUEDBOSNKexNQQH0/NV66GVBAH/RsVn0OUECvJeSDnhFRQLsnDwu1slBArthfdk9CUEChZ7PqczFQQNJvXwfOSVBAW0I+6NkcUUDVCWgibNRQQFwgQfFjHFFA9pfdk4cpUUC2hHzQsz1RQOlILv8hEVFAbjSAt0AKUUDwhclUwQxRQNv5fmq8+FBAKe0NvjDJUECJ0t7gC6NQQHZxGw3gBVFA7Q2+MJnmUEDcaABvgRBRQL6fGi/dAFFAcF8HzhnpUECM22gAb7FQQP5D+u3rpFBAgQTFjzG/UEB9rrZifwFRQOF6FK5HLVFAz2bV52o/UUB0JJf/kFZRQLWmeccp1lFAVHQkl/+gUUBKDAIrh15SQBQ/xty1rFJANqs+V1vBUkC4HoXrUZRSQM4ZUdobKFJALv8h/fYRUkAJih9j7jZSQOjZrPpcNVJAayv2l903UkDSAN4CCUpSQJzEILByWFJAL90kBoE9UkAc6+I2GmRRQHL5D+m3X1FABFYOLbK9UUCGyVTBqMhRQMrDQq1pSlFA6pWyDHFUUUBjf9k9eVBRQKwcWmQ7Z1FAXCBB8WN0UUCKjuTyHzpRQNqs+lxtaVFAY+5aQj7QUEBcIEHxYwxRQIGVQ4tsT1FAo5I6AU20UUBApN++DuxRQPmgZ7PqF1JAi/1l9+R1UkACmggbnmpSQHE9CtejdFJA5fIf0m+7UkBbsb/snuxSQGaIY13czlJAdZMYBFYqU0BCPujZrE5TQPvL7snDplJAwaikTkA7U0A1XrpJDIJTQCQofoy5f1NAf2q8dJNcU0AwKqkT0FhTQPhT46WbvFNAC7WmecepU0DKVMGopKZTQEa28/3UmFNAw2SqYFS2U0CNl24Sg4hTQOjZrPpcZVNAduCcEaX1UkC7Jw8LtXpTQIcW2c73m1NAnYAmwob3UkDo2az6XElTQHgLJCh+SFNAu7iNBvB2U0AZBFYOLcJSQD9XW7G/mFJA4umVsgxpUkDZPXlYqClSQLraiv1lY1JAMEymCkZFUkD8GHPXEoJRQN21hHzQu1FAYTJVMCrxUUAVjErqBERSQOhqK/aXVVJAufyH9NtLUkBiodY07whRQPVKWYY4alFAMzMzMzPvUEBQjZduEkdRQMiYu5aQt1FAysNCrWmeUUC0WfW52vpRQI4G8BZIvFFAPzVeukkEUkA1XrpJDFZSQDGZKhiVsFFAPE7RkVyqUUCcM6K0N+RRQDC7Jw8LGVJAB1+YTBU0UkBuowG8BSpSQOqVsgxxOFJAufyH9Ns3UkBvEoPAypFSQFg5tMh26lJAqaROQBM5U0CCc0aU9jpTQMBbIEHxS1NA0LNZ9bmiU0B+HThnRNFTQPhT46WbCFRAg8DKoUUOVEB0JJf/kH5TQEw3iUFgVVNAY+5aQj5oU0D0/dR46UZTQDxO0ZFcAlNAH/RsVn0WU0AGEhQ/xhxTQE7RkVz+P1NAVHQkl/98U0Ag0m9fBz5TQERpb/CFNVNA0SLb+X4qU0Am5IOezQJTQH9qvHSTNFNAat5xi

In [19]:
p.segment(df.seqs[dec], df.high[dec], df.seqs[dec], df.low[dec], color='red')

bokeh.models.renderers.GlyphRenderer(
    id='c058c49d-cbe2-4675-ac30-76440204496b',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='3a3732c1-e7a1-4fa7-9b0f-bd5c94e2ef1a',
        callback=None,
        data={'y0': {'__ndarray__': 'BTQRNjytRkBfB84ZURJHQKqCUUmdQEhA3gIJih/DR0DNO07RkWxHQNk9eVioTUdAUkmdgCYyR0BvgQTFj7FGQGaIY13cvkZAjNtoAG95R0B90LNZ9VlIQHDOiNLeeEhA2IFzRpSeSEA3GsBbILFIQMx/SL99vUhAeekmMQgESEAg0m9fByZIQGEyVTAqMUhATYQNT68MSEBYObTIdoZIQPT91HjpPkhA+FPjpZuESEBkO99Pjc9IQIQNT6+UjUhAU5YhjnWpSEADeAskKIZIQMnlP6TfZkhARGlv8IXhSEDuWkI+6HlJQFioNc07ZkpA001iEFjJSkBbQj7o2dxKQBdIUPwY60pA9pfdk4flSkDgLZCg+ClLQFD8GHPX8kpA8rBQa5ofS0DfT42XbhJLQOeMKO0NfkxAayv2l90bTEBa9bnainVLQBKlvcEX3kpAPujZrPq0SkArhxbZzldLQJZDi2zne0pApSxDHOtaSkByio7k8t9KQP5l9+RhsUtAhxbZzvebS0CxUGuad0RLQAkbnl4pO0tAVn2utmKfS0AZBFYOLaJLQFYOLbKdJ0tA9UpZhjheS0BHcvkP6W9LQLivA+eMmEtA3+ALk6mSS0C8BRIUP4ZLQMPTK2UZKktAescpOpJrS0Djx5i7lghLQKpgVFInsEpA/7J78rA4SkA3GsBbIHFKQNk9eVioPUpAn82qz9U2SkBMpgpGJSVKQAaBlUOLJElATDeJQWD1SEAs1JrmHVdJQKH4MeauPUhAT0ATYcOjSUCTqYJRSX1JQCSX/5B+g0lAQYLix5izSECn6Egu/xlJQDojSnuDF0lA0LNZ9bm6SUDKVMGopHZJQArXo3A9iklA8BZIUPyISUDA7J48LAxKQBrAWyBBOUpAxY8xdy0xSkCitDf4wuRJQGpN845T/ElA/Yf029f5SUDtnjws1OJJQHE9CtejwElAnl4pyxCfSUAzxLEubsNJQHxhMlUwwklAtMh2vp+SSUBd/kP67XtJQNBE2PD0KkpAvJaQD3pWSkBhw9MrZRlLQCEf9GxW9UpAMzMzMzO7SkDZX3ZPHgZLQAIrhxbZLktAbcX+snsaSkBvEoPAyjFKQI0o7Q2+6ElA7Z48LNTiSUCKsOHplcpJQNqs+lxtrUlAEqW9wRemSUCV1AloIvRJQISezarPZUpA2ht8YTJlSkDGbTSAt3BKQNbFbTSAP0pA/yH99nUoSkAxmSoYlUxKQHo2qz5XS0pA9+RhodYUSkDJdr6fGjdKQIJzRpT2XkpAj+TyH9I/SkC8dJMYBHZKQCZTBaOSikpAPL1SliFmSkAukKD4MQZKQH0/NV66EUpACKwcWmQDSkCfzarP1e5JQPRsVn2urklAb4EExY/RSUDIBz2bVX9JQMpUwaikRklAzF1LyAcNSUC0yHa+n5JIQJ0Rpb3BB0pA9bnaiv0NSkCKH2PuWpJKQL7BFyZTtU5AsVBrmnfETkBZhjjWxZ1OQLprCfmgr05AwTkjSnuTTkDnHafoSEZPQBPyQc9mHU9AcM6I0t6QT0DSAN4CCaJPQFafq63Y909AOPjCZKowUEDOqs/VViRQQG40gLdAHlBAkst/SL+NT0COBvAWSCBQQArXo3A9alBAKVyPwvVQUECitDf4wixQQGdEaW/wSVBAM8SxLm4PUEBEi2zn+8lPQPs6cM6IHlBAjnVxGw3wT0APC7WmeTNQQMuhRbbzhU9AppvEILBiT0AE54wo7cVPQPd14JwRXU9AVp+rrdjvTkDWxW00gNdOQDBMpgpGEVBAH/RsVn0+T0ACvAUSFJtQQB4Wak3zWlBAVg4tsp1TUEBSJ6CJsKFQQJkqGJXUlVBAn6ut2F+GUEC5/If023dQQArXo3A9slBAwOyePCy4UEBvEoPAyv1QQFfsL7snx1BArIvbaADHUECx4emVsrxQQH9qvHSTdFBAEhQ/xtxNUEDUmuYdp2RQQGEyVTAqXVBAbAn5oGdjUEAXSFD8GKNQQLU3+MJkclBAW9O84xQtUEA9LNSa5iVQQC6QoPgx9k9Ao5I6AU1MUEAjSnuDL0RQQDsBTYQNS1BAT0ATYcPzT0BkO99Pjc9PQMHKoUW2o1BAodY07zihUECjAbwFErxQQPvL7snDdlBA+8vuycN2UED9h/Tb1yFQQFInoImwQVBAldQJaCIAUUD/If32ddRQQO/Jw0KtRVFA+zpwzoguUUBR2ht8YRJRQB6n6Egu+1BAqFfKMsQBUUCASL99HfhQQF5LyAc9+1BAfoy5awnlUEBkO99Pja9QQE9AE2HDG1FAoBov3SQKUUB6Nqs+V9tQQP32deCc9VBA+FPjpZvwUEBzaJHtfPdQQPAWSFD89FBA1lbsL7vzUEDy0k1iELxQQIZa07zjfFBAcRsN4C2wUEAE54wo7alQQHsUrkfhclBA2T15WKiZUEB56SYxCCxRQN/gC5OpzlFAwaikTkCnUUCPwvUoXMtSQIXrUbgeQVJAGeJYF7exUkA7cM6I0pZSQJzEILBynFJAqMZLN4ltUkBKDAIrh15SQEvIBz2bJVJAeAskKH4EUkBYObTIdpJRQLgehetRpFFAio7k8h86UUDD9Shcjx5RQD2bVZ+rZVFAjnVxGw0cUUCMuWsJ+WBRQKRwPQrXd1FA3bWEfNBTUUCJ0t7gC7NRQOqVsgxxVFFAYTJVMCpJUUBhMlUwKn1RQKwcWmQ7M1FAtRX7y+5RUUCUh4Va08BRQLUV+8vuIVJAEHo2qz5rUkCaCBueXtFSQCo6kst/1FJA1lbsL7v/UkB8YTJVMCJTQLraiv1lQ1NA2T15WKhJU0D0/dR46UpTQOF6FK5HeVNAOUVHcvkvU0BNFYxK6qxTQFafq63YZ1NAcvkP6bdvU0Bbsb/sntxTQE7RkVz+21NAyjLEsS7OU0DJ5T+k36JTQP5l9+RhpVNAylTBqKSmU0DChqdXyo5TQOtztRX7u1NAqMZLN4mNU0AZ4lgXt+lSQCbkg57NrlNACD2bVZ8rU0BI4XoUrv9SQMdLN4lBTFNAAwmKH2NqU0DuWkI+6FFTQLx0kxgEulJAImx4eqVEUkDOiNLe4HtSQILix5i74lFA3gIJih/nUUCcM6K0N+BRQI51cRsNuFFAUwWjkjrFUUBSJ6CJsJ1RQCGwcmiRMVJAwOyePCxMUkCqYFRSJwxSQAdfmEwVLFFAHThnRGmXUUA5tMh2vr9RQJ88LNSa9lFAldQJaCIoUkCBBMWPMSdSQE+vlGWIU1JAt2J/2T1NUkCTOgFNhD1SQOVhodY0C1JAmSoYldSRUUDnHafoSHZRQPVKWYY4ilFAysNCrWmeUUAW+8vuyRdSQF8pyxDHPlJAjNtoAG/lUkCpE9BE2DxTQBWMSuoECFNAarx0kxgoU0B/2T15WFBTQBiV1AloWlNASgwCK4fSU0Bd/kP67RNUQKpgVFInEFRAyjLEsS5eU0Ch1jTvOHlTQESLbOf7KVNATtGRXP4/U0CutmJ/2YVTQF1txf6yQ1NAaQBvgQQtU0DarPpcbTFTQNc07zhFm1NAIGPuWkKKU0BrK/aX3adTQOwvuycP71NAAU2EDU/fU0C62or9ZbtTQOXQItv5ZlNAAiuHFtlGU0Cn6Egu/0lTQKabxCCwMlNAu7iNBvAiU0A730+Nl1pSQM6I0t7g+1JA3NeBc0awUkCoNc07TmVSQKW9wRcm/1JAh6dXyjIkU0CoNc07TmFTQGdEaW/wXVNAidLe4

In [20]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='green', line_color='green', source=sourceInc)

bokeh.models.renderers.GlyphRenderer(
    id='59363bf3-1e5d-4105-b408-5af2372f2fdb',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='5823f253-1847-42bb-be95-b27e92807231',
        callback=None,
        data={'height': {'__ndarray__': 'AFBAE2HDkz8A2j15WKi1P4C5awn5oNM/QKkT0ETY4D8AFR3J5T/UPwDeAgmKH+s/AFYOLbKd4T8AKVyPwvXoPwCEwMqhRaY/QOF6FK5H6z+ArwPnjCjRPwCq8dJNYtA/APw6cM6Isj8A5/up8dLdP4DplbIMcdw/AHpYqDXNuz+AzojS3uDnPwCDwMqhReo/ACQofoy55z+AJeSDns3oP8Bm1edqK+Q/ADQRNjy92j8AfNCzWfWpP4DLf0i/fec/AHJoke18xz8AfNCzWfWpP4CXbhKDwN4/AG7whclUsT9ADwu1pnnjPwBe3EYDeMM/gLG/7J486D8AXEvIBz2rPwDTTWIQWOE/ADC7Jw8LtT+Agy9MpgriPwAKRiV1Aro/ABcmUwWjyj8APE7RkVy+PwBHcvkP6c8/ACNKe4Mv3D8ALm6jAbzNP4CDL0ymCvI/gKMBvAUS5j/AA+eMKO3xP0Aofoy5a+E/AISezarP0T+AHOviNhrQPwAe6+I2GrA/ADQRNjy90j8A3LWEfNCzP4Au/yH99tE/AAHeAgmKxz8At0CC4sfIPwDc14FzRu4/ACNKe4MvzD8AzhlR2psDQADSkVz+Q8I/QBIUP8bc6T8AnIAmwoa3P0DY8PRKWeI/AKabxCCw5j+AW7G/7J7YPwCKsOHpldI/AC2yne+nxj+AApoIG57SPwBQ/Bhz19o/gLPqc7UV4z8ABoGVQ4vMPwAQNjy9UpY/AGiz6nO1tT8AuECC4seYPwCmLEMc68I/gLPqc7UV0z8AGCZTBaOiPwCyne+nxts/AAdfmEwV3D8ARrbz/dS4PwCze/KwUMs/wNy1hHzQ5z8AKVyPwvXmP4Cs+lxtxdI/AJy7lpAPqj/AVMGopE7iPwAYJlMFo6I/ALMMcayLwz+AbhKDwMrdPwA41sVtNMg/AGiz6nO1tT8AyeU/pN/ePwCcM6K0N+o/APVKWYY42j+AwvUoXI/ePwCzDHGsi8M/APrt68A5wz8AyCCwcmiRP0AawFsgQeE/AFg5tMh2zj+AMsSxLm7TPwAOvjCZKuA/gF3cRgN41z8AlIeFWtPEPwBU46WbxLA/ANgrZRnimD8APnlYqDW9P4AyxLEubtM/APfkYaHWxD8AcM6I0t7IPwA+eVioNb0/ADMzMzMz2z+AJsKGp1fSPwBrK/aX3d8/gHVxGw3g4T+Ae4MvTKbSPwCKY13cRrM/AD/G3LWExD8AidLe4AvbP4C4HoXrUdA/QMsQx7q44T8AeDarPlebPwAPnDOitM8/AITrUbgetT8AlIeFWtPEPwB9PzVeusE/gFFJnYAm3j8AS+oENBHGPwAbDeAtkNg/ACgxCKwcqj8APizUmua9PwBgKcsQx7o/gNEA3gIJ5j8AHKfoSC6vPwBTBaOSOtk/ADws1JrmrT8AQK5H4XpkP4Bp3nGKjtQ/gGsJ+aBn2z+AzojS3uDlPwBGJXUCmsg/APhT46Wb4j8AAG+BBMW/P4D9ZffkYdk/AOauJeSDvj8ALSEf9GzGP8DoSC7/IeE/AC4hH/Rsxj8APSzUmubNP6AMcayL2/A/oApGJXUC9z+ASuoENBHSP8BK6gQ0EeI/gHJoke186z8ADCQofoy5PwCEns2qz7U/ADEIrBxa4D+gqYJRSZ3/P8DYX3ZPHvA/AO84RUdywT+ArrZif9npP2DuWkI+6PQ/APwYc9cSwj/AQ4ts5/vhPwCbd5yiI8E/ACNKe4Mv6D+AJXUCmgjTPyClvcEXJvA/AHbgnBGl3T8AIR/0bFbdPwBDHOviNso/AIqw4emV3j8A9GxWfa7aP0DD0ytlGfI/ANgbfGEypT8AOt9PjZe+P8ArZRniWPM/gMPTK2UZ4j8AXP5D+u3bPwCoHFpkO68/gLDh6ZWy4D8A6Gor9pfZP0CEDU+vlOM/ANbFbTSA0z+Axyk6ksvnPwDek4eFWsM/AGgibHh60T/ABRIUP8bgPwAiH/RsVr0/wB/Sb18H6D9gQj7o2az0P8A+V1uxv/E/AJ6AJsKGzz8A78nDQq3pPwCb5h2n6O4/AAAXSFD8iD8AZapgVFLbPwBiMlUwKsE/ADj4wmSq2D8Aarx0kxjMP4DiWBe30eg/AA3gLZCg4D+AwFsgQfHrPwDqc7UV+8M/AFMFo5I61T8AgBbZzveTP4CC4seYu9I/ANNNYhBY3T8Ap3nHKTraPwAwuycPC+M/AKABvAUShD8AiBbZzvezPwDMoUW2870/gOpztRX70z/g52or9hcBQADgLZCg+N0/QBNhw9Mr4T8A1lbsL7vHP4BIv30dOO8/QCNKe4MvAkAAGlHaG3zBP0B4eqUsQ/A/AHh6pSxD0D8AoAG8BRKUPwBkXdxGA9Q/ALx0kxgE5D8AGIxK6gS0PwAIrBxaZNM/ABiMSuoEtD8A4QuTqYLRP8Dh6ZWyDPY/ADojSnuD4T8A+8vuycPiPwBjXdxGA9Q/APAWSFD8qD8ASAwCK4e2PwBAE2HD0+E/AOtRuB6F2z8AE/JBz2bdPwDGSzeJQcA/AOILk6mCwT8AINJvXwfeP0A1XrpJDPY/AFAXt9EAnj8A61G4HoXbPwB/arx0k/A/AHuDL0ym3j8A5q4l5IPGP4B+arx0k+A/AEku/yH95D8AFtnO91PfPwAep+hILtc/AFInoImw2T8AXgfOGVHKPwBER3L5D8k/ADhnRGlv2D8gBFYOLXIQQEBCPujZrPQ/AOJYF7fR3D8AEce6uI3WPwBwzojS3uY/ANV46SYx7D8AcAKaCBueP4ClLEMc6+A/AN5GA3gLxD8AZqpgVFLPP4A+V1uxv+g/AB3J5T+k7z8AqTXNO07VPwDswDkjSv0/QGpN845T8j+ATBWMSurgPwDVeOkmMew/wNeBc0aU8z+ApSxDHOvgPwArhxbZztc/gEgu/yH95D+A8rBQa5rwPwCfPCzUmuA/AERHcvkPuT9A7zhFR3LwPwBQF7fRAJ4/YIEExY+xBUCABvAWSFDqP4AZ4lgXt/w/AKA8LNSatj9Az2bV52rxP0BUUiegifc/gNLe4AuT4T8AcF8HzhnJPwAHzhlR2tc/QBZqTfOO/D8AoDws1Jq2PwCKH2PuWso/gBov3SQG5T8AWH2utmLPPwAvbqMBvNk/ABTQRNjw0D8Aio7k8h/ePwCYbhKDwN4/AO58PzVeyj8AoAG8BRKUPwDF/rJ78tA/AJDtfD81nj8Az2bV52rpPwDYX3ZPHs4/gGZmZmZm5D+A7Q2+MJnmPwA8TtGRXOo/AHACmggbnj9gB84ZUdoPQADtnjws1NI/gFKWIY514z/A6ZWyDHH+PwAEVg4tsu8/AHo2qz5X5T+A8IXJVMHuPwAXt9EA3tI/AJYhjnVx3z+A4xQdyeXnP4AXt9EA3uI/QJT2Bl+Y9D8AwFsgQfH5PwB8YTJVMN4/ANBE2PD05j8AXW3F/rLlPwAwuycPC+s/AEvIBz2b0T8A5BQdyeXHPwAjSnuDL+4/ANcS8kHP3j+A/WX35GH1PwA9m1Wfq+s/APd14JwR9T/Ap1fKMsTxPwBX7C+7J+k/gP1l9+Rh5T+Aqs/VVuzhPwDUmuYdp/M/AD7o2az60D/A9NvXgXPxPwAwdy0hH7Q/ALx0kxgE1j8A6pWyDHHkP4AqGJXUCeA/AJayDHGsyz8AiUFg5dDePwA4+MJkquA/gPs6cM6I6D8ALLKd76e2PwAmUwWjkug/gJtVn6ut5j8AoAG8BRKEPwAbnl4py9Q/Q

In [21]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='red', line_color='red', source=sourceDec)

bokeh.models.renderers.GlyphRenderer(
    id='ddc38da4-1bd4-46c8-9c9d-a3349e3ea6f0',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='fe2472be-2869-4ebf-98d2-637e7448129d',
        callback=None,
        data={'height': {'__ndarray__': 'AMWPMXct1b+A1AloImzYv4CJsOHpldq/AHGsi9to5r+ALSEf9GzavwBIQBNhw5O/ACuHFtnO6b8Afoy5awnZv4CQD3o2q+y/gNk9eVio2b8ASL99HTjPvwCoNc07TpG/AGkAb4EExb8A5h2n6Ei+v4CjAbwFEtC/ALpJDAIrt7+AKO0NvjDpv4DuWkI+6O2/AP+ye/KwwL+AeHqlLEPgv4ClLEMc6+a/ADXvOEVHwr+Axks3iUHQvwCSy39Iv72/gM2qz9VW3L8ALEMc6+Kmv4ClLEMc69a/AKs+V1ux178A4C2QoPixvwBJv30dOMe/AJT2Bl+Y1L8AcayL22jAvwA17zhFR8K/AOm3rwPn1L+AYcPTK2XZvwD2udqK/cW/AFyPwvUo6L+A9bnaiv3Vv8CI0t7gC+O/QL99HThn6r/AuriNBvDzv2B7gy9Mpva/gN7gC5Op3r9AYOXQItvyvwCh+DHmrsW/wADeAgmK57+APldbsb/UvwBgdk8eFrq/gMnlP6Tf3r8AyxDHurjFvwCCc0aU9ra/gEs3iUFg0b8AOGdEaW/Av4Bc/kP67du/gCBB8WPM5b8AAFUwKqljvwCgAbwFEnS/oH77OnDO9r8AHcnlP6Tlv8D8h/Tb1+G/oPpcbcX+8b/AudqK/WXtvwBvgQTFj/C/QFRSJ6CJ4r9A/yH99nXivwAgQfFjzJ2/ADnWxW00yL/g8PRKWYb+vwBHA3gLJOq/AEDG3LWErL8Ax7q4jQb6v8Bm1edqK+S/QK1p3nGK4r8AgnNGlPbWvwBGtvP91Li/gJT2Bl+Y0L8AFK5H4XrUvwBpb/CFydS/gEW28/3U1L+Aarx0kxjYvwDUmuYdp+K/ALlrCfmgx78A9ihcj8LFv4C4HoXrUdC/APCFyVTB3L8A2IFzRpTavwBANV66Scy/AIhjXdxGs78AiWNd3EbDvwDSAN4CCcK/AJZDi2znw7+AdXEbDeDRv4BkO99Pjd+/gKyL22gA178AidLe4AvLvwBgK/aX3YO/gA8LtaZ5178AVOOlm8Swv0C9UpYhju+/AIhjXdxGs78AbqMBvAWyvwCGp1fKMsS/ANQrZRniqL8A63O1FfvLv8DHmLuWkOG/AKFns+pzxb8AS1mGONbNvwBm9+RhoeC/AJb2Bl+YvL/AGi/dJAblvwCPwvUoXMe/AJghjnVxm78An6ut2F/SvwCa5h2n6Li/gABvgQTF378ATKYKRiWlvwBR2ht8YdK/AKibxCCwor8ASFD8GHPPv4DBqKROQNu/ALWmeccpyr+A3nGKjuTWv4AtIR/0bNa/AO7rwDkj6L+ALSEf9GzWv8Bc/kP67em/ACcxCKwc5r8A/0P67evIv8DLf0i/feW/gOtRuB6F8b8AhslUwajmvwAIrBxaZKu/AF8HzhlR2r8A+1xtxf7Cv0BN845TdOK/AKibxCCwor8AUiegibDZvwDImLuWkNu/AGUZ4lgX178AwFsgQfHhvwBALNSa5o2/AGQZ4lgXt78AFD/G3LXMvwAOnDOitLe/AJghjnVxq78A1JrmHae4vwAhH/RsVs2/AChcj8L1uL9A/Bhz1xLgv0AawFsgQe2/AAmKH2Pu1r8An6ut2F/avwCegCbChs+/AKrx0k1i0L8AYjJVMCrBvwA2PL1Slum/QAfOGVHa8L/Aaiv2l93hv8Af0m9fB+i/AI/k8h/Sz7+AK2UZ4ljTvwBbQj7o2dC/wFz+Q/rt678AeJyiI7msvwCamZmZmbm/wPyH9NvX5b+gSuoENBH4v2B2Tx4WavW/AIPAyqFFzr+AhVrTvOPwvwAtIR/0bN6/gAgbnl4p4b8AMQisHFrovwCK0t7gC8O/AGSqYFRS278A5h2n6EjOvwD+Q/rt69S/gGWIY13c4L8A+THmriXqvwBg5dAi29W/AAgbnl4py7+AUiegibDvv4AW2c73U/G/AF7cRgN4y7+ALm6jAbztv4C84xQdyeu/AIAW2c73k7+AMlUwKqntv4BWfa62YvW/ALuWkA961r8AzKFFtvOtvwANcayL29C/AACRfvs6wL8AYOXQItvVv4CI0t7gC+O/AHUCmggb4r8AGlHaG3zRvwDgLZCg+N2/AJhMFYxK5L8A7uvAOSP0v4AMcayL2+C/gHA9Ctej8b8A001iEFjdvwDgLZCg+N2/AGgibHh60b8AuNEA3gK5vwB/arx0k+C/AGLD0ytl3b8A6K4l5IPGv4CuR+F6FOi/AMNkqmBU5L8AgBbZzveTvwAJrBxaZNO/ANGRXP5D2r8A5BQdyeXfvwBYF7fRAK6/ALjRAN4Cub8A/MvuycPCvwCY3ZOHhda/AO7rwDkj4r8ABhIUP8bMvwA6kst/SM+/gKdXyjLE578AufyH9NvzvwAAb4EExce/AEgMAiuHtr8AfR04Z0TVvwDhC5OpgtG/gPvL7snD4r8AsAxxrIu7v4BZhjjWxem/APgx5q4l3L+A1edqK/b+vwCDwMqhRdq/wOI2GsBb8b8AeXqlLEPQv0CC4seYu/C/QDeJQWDl+r8A5BQdyeXfvwD5MeauJdy/AD/G3LWE9L8wl/+QfnsSwAB6pSxDHOO/ANV46SYx3L8AjnVxGw3UvwB4jLlrCam/APJjzF1LwL8AFmpN847Lv4BjXdxGA/K/gGPuWkI+5r8A9wZfmEzVvwBwApoIG56/ALQMcayLu78AF9nO91Pfv+Ai2/l+agfAAEhHcvkPub8ADCQofozBvwBwApoIG66/ADVeukkM5L8AEce6uI3WvwAep+hILve/ADlnRGlv2L8AJLn8h/TfvwDcRgN4C7S/QGDl0CLb8b8AsAG8BRKUvwBSJ6CJsOm/ABHHuriN1r+AGi/dJAblvwBz1xLyQeG/AAg9m1Wfu78AY+5aQj7cvwAijnVxG9m/AD29UpYh3r8ACD2bVZ/Lv8C7lpAPevC/gMKGp1fK6L8AwoanV8rwvwDgvg6cM+K/AN21hHzQ7b+ACBueXinwv8C2Yn/ZPfS/QPW52or9+r8ACAIrhxapvwCcPCzUmra/gJT2Bl+Y5r+AgCbChqflvwDtDb4wmda/QOauJeSD8r/A8PRKWQYBwABAguLHmLu/wGSqYFRS+L+ABaOSOgHwvwDsnjws1NK/AMKGp1fK8L+g4xQdyWUAwAAMAiuHFrm/ADxO0ZFc2r9ABFYOLbL/v4DysFBrGgHAAKut2F92078AkX77OnDkvwBX7C+7J+m/gPUoXI/C778Ab4EExY/dvwD+1HjpJsm/AKvP1Vbs4b8A46WbxCDUv4AgQfFjzPa/AJayDHGs678A5KWbxCDUvwCwcmiR7ey/AJiyDHGsu78A/fZ14JzRvwAAeAskKK6/APz2deCcwb8A/PZ14JzBvwCwlGWIY8W/AOI2GsBbwL8AKO0NvjCpvwA+6Nms+uC/ALCUZYhjxb+A3SQGgZXnv4BqTfOOU+a/AJ7vp8ZL4b8A93XgnBHlvwD99nXgnNG/AMoyxLEuzr9APujZrPr8v4DLEMe6uOu/gM07TtGR6L8AvsEXJlPhvwBz+Q/pt9u/AI1K6gQ02b+APujZrPrsvwDwp8ZLN7m/gCO5/If04b8AkKD4MebivwDaG3xhMtm/gMX+snvy9L8ACtejcD3uvwAK16NwPd6/gFRSJ6CJ8L8A/yH99nXqv2CR7Xw/NQbAAMDsnjwslL8AIGPuWkKuvwCDwMqhRdK/AMDKoUW2y78A54wo7Q3WvwDA7J48LKS/A

In [24]:
bk.save(p, './static/candle_stick.html', title='5yr_candlestick')

/Users/sjschmidt/.local/share/virtualenvs/stock_viz-8nQnKl7U/lib/python3.6/site-packages/bokeh/io/saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/Users/sjschmidt/CodeFellows/401/python/seattle-python-401d9/class-17-data-viz/demos/stock_viz/static/candle_stick.html'

In [ ]:
# bk.show(p)